This code takes a data output from Looker and determines order to first offer metrics for Tread+ CPSC reporting

In [1]:
response1 = input('Enter the year and month of reporting in the format YYYY-MM: ')

Enter the year and month of reporting in the format YYYY-MM:  2024-03


In [2]:
import pandas as pd

In [3]:
#Read in data from Looker csv export
df = pd.read_csv('Tread+ OTFOD Raw Data.csv',encoding='iso8859_1')

/var/folders/vm/78_qysq11pj_svvr0pc6qz6h0000gp/T/ipykernel_6658/3921277969.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Tread+ OTFOD Raw Data.csv',encoding='iso8859_1')


Data Cleaning

In [4]:
#Drop looker index row
df = df.drop('Unnamed: 0',axis=1)

In [5]:
#Convert OTD column to a float for comparison with first available OTD
df['OTD'] = df['OTD'].str.replace(',','').astype(float)

In [6]:
#Create boolean fields to check if OTD values are blank or not
df['FA OTD Bool'] = df['First Available OTD'].isna()
df['OTD Bool'] = df['OTD'].isna()

Formulas

In [7]:
#Define a formula to calculate OTFOD depending on values in boolean fields
def OTFOD_values(row):
    if row['FA OTD Bool'] == False and row['OTD Bool'] == False and row['First Available OTD'] < row['OTD']:
        return row['First Available OTD']
    elif row['FA OTD Bool'] == False and row['OTD Bool'] == False and row['First Available OTD'] >= row['OTD']:
        return row['OTD']
    elif row['FA OTD Bool'] == True and row['OTD Bool'] == False:
        return row['OTD']
    else:
        return row['First Available OTD']

In [8]:
#Define a formula to compare both booleans and exclude them if both are blank
def OTFOD_comparison(row):
    if row['FA OTD Bool'] == True and row['OTD Bool'] == True:
        return 'Exclude'
    else:
        return 'Include'

In [9]:
#Filter dataset to create specific return & relocation filters, save each as new dataframe
df_return = df.loc[(df['External Product Type'] == 'tread-plus') & (df['Type'] == 'return')].copy()
df_relo = df.loc[df['Slug'] == 'room-to-room-tread-plus'].copy()

Return data processing

In [10]:
#Use a lambda function to apply the formula along each row in the dataframe and save results in a new column
df_return['OTFOD Final'] = df_return.apply(lambda row : OTFOD_values(row),axis = 1)

In [11]:
#Use a lambda function to apply the formula along each row in the dataframe and save results in a new column
df_return['Status'] = df_return.apply(lambda row: OTFOD_comparison(row),axis=1)

In [12]:
#Perform OTFOD calculations
sub22 = ((df_return['Status'] == 'Include') & (df_return['Created At Month'] == response1) & (df_return['OTFOD Final'] < 22)).sum()
available = ((df_return['Status'] == 'Include') & (df_return['Created At Month'] == response1)).sum()
rate = (sub22 / available) * 100

Room to room data processing

In [13]:
#Use a lambda function to apply the formula along each row in the dataframe and save results in a new column
df_relo['OTFOD Final'] = df_relo.apply(lambda row : OTFOD_values(row),axis = 1)

In [14]:
#Use a lambda function to apply the formula along each row in the dataframe and save results in a new column
df_relo['Status'] = df_relo.apply(lambda row: OTFOD_comparison(row),axis=1)

In [15]:
#Perform OTFOD calculations
relo_sub22 = ((df_relo['Status'] == 'Include') & (df_relo['Created At Month'] == response1) & (df_relo['OTFOD Final'] < 22)).sum()
relo_available = ((df_relo['Status'] == 'Include') & (df_relo['Created At Month'] == response1)).sum()
relo_rate = (relo_sub22 / relo_available) * 100

Final Values

In [16]:
print('OTFOD < 22: ', sub22)
print('OTFOD Available: ', available)
print('% within 21: ', round(rate,2))
print('Relo OTFOD < 22: ', relo_sub22)
print('Relo OTFOD Available: ', relo_available)
print('Relo % within 21: ', round(relo_rate,2))

OTFOD < 22:  44
OTFOD Available:  46
% within 21:  95.65
Relo OTFOD < 22:  13
Relo OTFOD Available:  14
Relo % within 21:  92.86


In [17]:
print('OTFOD Available: ', available + relo_available)
print('OTFOD < 22: ', sub22 + relo_sub22)
print('X exceeded 21 days: ', ((available + relo_available)-(sub22 + relo_sub22)))

OTFOD Available:  60
OTFOD < 22:  57
X exceeded 21 days:  3
